In [1]:
import networkx as nx
import pandas as pd

data = pd.read_csv("../Datasets/Dataset.csv")

In [2]:
import pandas as pd
import torch
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx
import networkx as nx

# Initialize a NetworkX graph
G = nx.Graph()

# Step 1: Add nodes with unique types
node_types = ['campaign_item_id', 'advertiser_id', 'ext_service_name', 'creative_id', 
              'search_tags', 'template_id', 'channel_name', 'keywords']

# Add nodes to the graph
node_mapping = {}  # To map node names to unique indices
current_index = 0
for col in node_types:
    for value in data[col].unique():
        node_name = f"{col}_{value}"
        G.add_node(node_name, type=col)
        node_mapping[node_name] = current_index
        current_index += 1

# Step 2: Add edges based on relationships
for index, row in data.iterrows():
    G.add_edge(f"campaign_item_id_{row['campaign_item_id']}", f"advertiser_id_{row['advertiser_id']}")
    G.add_edge(f"creative_id_{row['creative_id']}", f"search_tags_{row['search_tags']}")
    G.add_edge(f"ext_service_name_{row['ext_service_name']}", f"channel_name_{row['channel_name']}")
    G.add_edge(f"campaign_item_id_{row['campaign_item_id']}", f"creative_id_{row['creative_id']}")

# Optional: Add weighted edges
for index, row in data.iterrows():
    G.add_edge(f"campaign_item_id_{row['campaign_item_id']}", f"ext_service_name_{row['ext_service_name']}", weight=row['media_cost_usd'])

# Step 3: Generate node features
# Here, we'll create dummy features. Replace this with actual node features if available.
num_features = 8  # Number of features per node
features = torch.rand(len(node_mapping), num_features)

# Assign features to nodes
for node, idx in node_mapping.items():
    G.nodes[node]['feature'] = features[idx]

# Step 4: Convert the NetworkX graph to PyTorch Geometric's format
data = from_networkx(G)

# Step 5: Add node features to PyG data
data.x = features  # Node features
data.edge_attr = torch.tensor([G[u][v].get('weight', 1.0) for u, v in G.edges()], dtype=torch.float)  # Edge weights

# Check the PyG data object
print(data)


ModuleNotFoundError: No module named 'torch_geometric'